In [146]:
from coinbase.wallet.client import Client
import pandas as pd
import requests
from datetime import datetime as dt, timedelta

In [147]:
api_key = "abRJTjZ5f940NqDm"
api_secret = "iZUizAftYAExBKIFktworjDksAWajsHU"
client = Client(api_key, api_secret)

In [148]:
def get_candles(start, end):
    url = "https://api.exchange.coinbase.com"
    sym = "LINK-USD"
    barSize = "86400"
    timeStart = start.isoformat()
    timeEnd = end.isoformat()
    parameters = {
        "start":timeStart,
        "end":timeEnd,
        "granularity":barSize
    }

    data = requests.get(f"{url}/products/{sym}/candles", 
                        params=parameters, 
                        headers= {"accept": "application/json"})
    return data



In [149]:
def kline_to_df(start, end):
    kline_data  = eval(get_candles(start,end).text)
    headers = ["timestamp", "low", "high", "open", "close", "volume"]
    df = pd.DataFrame(data=kline_data, columns=headers)
    df["timestamp"] = df["timestamp"].astype(int)
    for i in range(len(df["timestamp"])):
        df["timestamp"][i] = dt.fromtimestamp(df["timestamp"][i])
    df.iloc[:] = df.iloc[::-1].values
    return df

In [ ]:
df1 = kline_to_df( dt(2022, 1, 2, 0, 0, 0, 0),dt(2022, 7, 1, 0, 0, 0, 0) )
df2 = kline_to_df( dt(2022, 7, 2, 0, 0, 0, 0),dt.now() )
df = pd.concat([df1, df2], axis=0)

In [152]:
df.to_csv('coinbase_link_hist_data.csv', index=False)